In [1]:
from pylabrobot.liquid_handling import LiquidHandler
from pylabrobot.liquid_handling import LiquidHandlerChatterboxBackend
from pylabrobot.visualizer.visualizer import Visualizer
from pylabrobot.resources.opentrons import OTDeck

from pylabrobot.resources.opentrons.load import *
from pylabrobot.resources.opentrons.plates import *

from pylabrobot.resources import set_tip_tracking, set_volume_tracking, set_cross_contamination_tracking
set_tip_tracking(True), set_volume_tracking(True)

# Optional, use when interested in protecting against accidental cross contamination
set_cross_contamination_tracking(True)

import pandas as pd
import csv
import os
import opentrons
import time

Golden mutagenesis is a technique that uses the modularity of golden gate cloning to generate diverse plasmid libraries. Libraries of DNA fragments can be assembled as modules to generate combinatorial diversity from a relatively small number of individual parts that can be assembled in a standardized way. 

This technique has been used to evolve a number of enzymes in recent years.

Further reading:
Golden Mutagenesis: An efficient multi-site-saturation mutagenesis approach by Golden Gate cloning with automated primer design
https://www.nature.com/articles/s41598-019-47376-1




In [2]:
# Make sure to use the ChatterBoxBackend() and the OTDeck()
lh = LiquidHandler(backend=LiquidHandlerChatterboxBackend(), deck=OTDeck())

await lh.setup()

vis = Visualizer(resource=lh)
await vis.setup()

Setting up the liquid handler.
Resource deck was assigned to the liquid handler.
Resource trash_container was assigned to the liquid handler.
Websocket server started at ws://127.0.0.1:2121
File server started at http://127.0.0.1:1337 . Open this URL in your browser.


In [3]:
from pylabrobot.resources import (
    corning_96_wellplate_360ul_flat,
    opentrons_96_tiprack_300ul,
)

Suppose you have three plates of DNA fragments corresponding to golden gate cloning modules. We are going to read in a worklist of golden gate assemblies using one fragment from each plate. Write a script that processes this worklist to pool the fragments into their respective wells.

"fragments_plate_1.csv", etc will contain data about the fragment contents in each plate.
"cloning.csv" will specify the desired result.

Write a script that uses PLR's built-in state tracker to verify if your robot script worked correctly. 

In [4]:
fragments_plate_1 = corning_96_wellplate_360ul_flat("Fragments_1")
lh.deck.assign_child_at_slot(fragments_plate_1, 4)

tip_rack = opentrons_96_tiprack_300ul("Tip_Rack")

lh.deck.assign_child_at_slot(tip_rack, 1)

fragments_plate_2 = corning_96_wellplate_360ul_flat("Fragments_2")
lh.deck.assign_child_at_slot(fragments_plate_2, 5)

fragments_plate_3 = corning_96_wellplate_360ul_flat("Fragments_3")
lh.deck.assign_child_at_slot(fragments_plate_3, 6)

pooling_plate = corning_96_wellplate_360ul_flat("Pooling_plate")
lh.deck.assign_child_at_slot(pooling_plate, 3)

Resource Fragments_1 was assigned to the liquid handler.
Resource Tip_Rack was assigned to the liquid handler.
Resource Fragments_2 was assigned to the liquid handler.
Resource Fragments_3 was assigned to the liquid handler.
Resource Pooling_plate was assigned to the liquid handler.


In [20]:

# Read the fragment CSV file
#os.chdir("PyLabRobot_Tutorials_BME590\Class Exercises\DNA_Fragments")
fragments_df = pd.read_csv("fragments_plate_1.csv")
print("First 5 rows of fragments_plate_1.csv:")
print(fragments_df.head())

# Read the cloning CSV file
cloning_df = pd.read_csv("cloning.csv")
print("\nFirst 5 rows of cloning.csv:")
print(cloning_df.head())

First 5 rows of fragments_plate_1.csv:
  well_id fragment_id
0      A1        X095
1      A2        X058
2      A3        X085
3      A4        X050
4      A5        X027

First 5 rows of cloning.csv:
  well_id    fragment_tuple
0     A01  (X076,Y150,Z216)
1     A02  (X030,Y127,Z280)
2     A03  (X078,Y110,Z217)
3     A04  (X020,Y138,Z231)
4     A05  (X083,Y119,Z219)


In [21]:
os.chdir(os.path.join(os.path.expanduser("~"),"PyLabRobot_Tutorials_BME590/Class Exercises/DNA_Fragments"))

def assign_fragments_from_csv(plate, csv_file):
    with open(csv_file, 'r') as f:
        reader = csv.DictReader(f)
        for row in reader:
            well_id = row['well_id']
            fragment_id = row['fragment_id']
            plate[well_id][0].set_liquids([(fragment_id, 100)])

assign_fragments_from_csv(fragments_plate_1, "fragments_plate_1.csv")
assign_fragments_from_csv(fragments_plate_2, "fragments_plate_2.csv")
assign_fragments_from_csv(fragments_plate_3, "fragments_plate_3.csv")

## Exercises
1. Write a script that pools the desired fragments in the pooling plate according to the worklist
2. Write a script that uses the state tracker to verify the contents of the pooling plate

Extra credit:
3. Find a way to recover from an error mid-way through your script without starting from the beginning. 